In [1]:
import polars as pl
from mimesis import Person, Address, Numeric
import xlsxwriter

In [2]:
peep = Person()
adrs = Address()
num = Numeric()

In [3]:
data = []
for _ in range(100_000):
    data.append({'name':peep.full_name(), 'address':adrs.address(), 'zip_cd':adrs.zip_code()
                 , 'state':adrs.state(), 'net_worth':num.integer_number(5000,50000)})

In [4]:
df = pl.DataFrame(data)

In [5]:
df.head(5)

name,address,zip_cd,state,net_worth
str,str,str,str,i64
"""Lynna Vang""","""1054 Miller Ce…","""84769""","""Tennessee""",26274
"""Moriah Harmon""","""1318 Jeopardy …","""68278""","""Pennsylvania""",36192
"""Kortney Burns""","""999 Saint Loui…","""99648""","""Ohio""",47950
"""Minna Fulton""","""477 Marvel Est…","""65209""","""Texas""",16183
"""Bobette Gallag…","""86 Dunsmuir Ru…","""31921""","""North Carolina…",12086


In [9]:
res = df.group_by(pl.col('zip_cd')).agg(pl.col('name').n_unique().name.suffix("_cnt")
                                        ,pl.col('address').count().name.suffix("_cnt")
                                        ,pl.col('net_worth').sum().name.suffix("_tot")
                            )
print(res)

shape: (63_091, 4)
┌────────┬──────────┬─────────────┬───────────────┐
│ zip_cd ┆ name_cnt ┆ address_cnt ┆ net_worth_tot │
│ ---    ┆ ---      ┆ ---         ┆ ---           │
│ str    ┆ u32      ┆ u32         ┆ i64           │
╞════════╪══════════╪═════════════╪═══════════════╡
│ 64934  ┆ 1        ┆ 1           ┆ 37000         │
│ 82576  ┆ 1        ┆ 1           ┆ 35236         │
│ 05687  ┆ 1        ┆ 1           ┆ 28699         │
│ 80119  ┆ 2        ┆ 2           ┆ 44029         │
│ 30331  ┆ 2        ┆ 2           ┆ 83101         │
│ …      ┆ …        ┆ …           ┆ …             │
│ 36826  ┆ 2        ┆ 2           ┆ 78335         │
│ 61734  ┆ 2        ┆ 2           ┆ 20762         │
│ 17027  ┆ 1        ┆ 1           ┆ 29030         │
│ 64088  ┆ 1        ┆ 1           ┆ 6443          │
│ 77732  ┆ 1        ┆ 1           ┆ 11726         │
└────────┴──────────┴─────────────┴───────────────┘


In [62]:
res.write_parquet("./agg_pl.parquet")
res.write_excel("./test.xlsx")